# CNN  CIFAR-10 by Densenet 

## Model funtion def
* Dnesenet struct by dnese layer → dense block → transition each block  and average pooling → Flatten → FC → outputs

* dense layer { 

* CIFAR10 → 10 calss, 10000 sample

In [1]:
#import library
import torch
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from torch import nn
from torch.autograd import variable
from torchvision.datasets import CIFAR10
from collections import OrderedDict
%matplotlib inline
sns.set()

In [4]:
#由小結構到大結構建立 dense layer - (Hx=[x1 + ... + xn]) - dense block - (transition) -

#dense_layer 為 組成 block 的單元體 (BN → Relu → conv2d(k=1,s=1) → BN → Relu→ conv2d(k=3,s=1))
class _dense_layer(nn.Sequential):
    def __init__(self, input_feature, growth_rate, bn_size, drop_rate):
        super(_dense_layer, self).__init__()
        self.add_module('BN1', nn.BatchNorm2d(input_feature)),
        self.add_module('Relu1', nn.ReLU(inplace=True)),
        self.add_module('Conv1', nn.Conv2d(input_feature, bn_size*growth_rate, kernel_size=1, stride=1, padding=1, bias=False)),
        self.add_module('BN2', nn.BatchNorm2d(bn_size*growth_rate)),
        self.add_module('Relu2', nn.ReLU(inplace=True)),
        self.add_module('Conv2', nn.Conv2d(bn_size*growth_rate, growth_rate, kernel_size=3,stride=3,padding=1,bias=False)),
        
        self.drop_rate =drop_rate
    #如果drop_rate >0 在 training 時 dropout x 變 feature
    def forward(self,x):
        feature = super(_dense_layer,self).forward(x)
        if self.drop_rate >0:
            feature = F.dropout(feature, p=self.drop_rate, training=self.training)
        return torch.cat([x,feature],1)

#建立 dense_block 
class _dense_block(nn.Sequential):
    def __init__(self, num_layers, input_feature, bn_size, growth_rate, drop_rate):
        super(_dense_block, self).__init__() #繼承 nn.sequential 
# 每過一層會累加Growth_rete
        for i in range(num_layers):
            layer = _dense_layer(input_feature + i*growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module(f'dense_layer{i+1}', layer)

    
#建立 block 與 block之間的 transition (BN → Relu → Conv2d → Avg_pool(k=2,s=2)  )
class _transition(nn.Sequential):
    def __init__(input_feature, output_feature):
        super(_transition, self).__init__()
        self.add_module('BN', nn.BatchNorm2d(input_feature)),
        self.add_module('Relu', nn.ReLU(inplace=True)),
        self.add_module('Conv1x1',nn.Conv2d(input_feature,output_feature,kernel_size=1,stride=1,bias=False)),
# Trasition layer 1.降低channel膨脹 2.Avg_pooling 保留背景訊息並降維(ps. max_pooling 由於保由最大值，保留圖片紋理訊息)
        self.add_module('Avg_pool', nn.AvgPool2d(kernel_size=2,stride=2))

#組合 DenseNet
class densenet(nn.Module):
    def __init__(self, growth_rate=32, block_of_layer=(6, 12, 24, 16),init_channel=64, bn_size=4, drop_rate=0, num_classes=1000):      
        super(densenet,self).__init__()
        #Data input first layer (Conv2d → BN → Relu → Maxpool(k=3,s=2,p=1))
        self.feature = nn.Sequential(OrderedDict([
            ('conv0',nn.Conv2d(3,init_channel,kernel_size=7,stride=2,padding=3,bias=False)),
            ('BN0',nn.BatchNorm2d(init_channel)),
            ('Relu0',nn.ReLU(inplace=True)),
            ('max_pooling',nn.MaxPool2d(kernel_size=3,stride=2,padding=1)),
            ]))
        num_features = init_channel
        #每個 Dence block中 layer數目
        for i, num_layers in enumerate(block_of_layer):
            block = _dense_block(num_layers=num_layers,input_feature=num_features,bn_size=bn_size,growth_rate=growth_rate,drop_rate=drop_rate)
            self.feature.add_module(f'Dense_block{i+1}',block)
            num_features = num_layers*growth_rate
            
            if i != len(block_of_layer)-1:
                trans = _transition(input_feature=num_features, output_feature=num_features//2)
                self.feature.add_module(f'transition{i+1}',trans)
                num_features = num_features//2
            
        #final
        self.feature.add_module('BN_f',nn.BatchNorm2d(num_features))
        self.feature.add_module('Relu_f',nn.ReLU(inplace=True))
        self.feature.add_module('Avg_pool_f', nn.AvgPool2d(kernel_size=7,stride=1))
        
        #flatter
        self.Classifier = nn.Liner(num_features,num_classes)
        
        #init variable
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal(m.weight.data)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Liner):
                m.bias.data.zero_()
    def forward(self,x):
        features = self.features(x)
        out = features.view(features.size(0),-1)
        out = self.Classifier(out)
        return out
            
#------------------------------------module------------------------------------#

In [5]:
model = densenet()

TypeError: __init__() got multiple values for argument 'input_feature'